<a href="https://colab.research.google.com/github/andivaanggoro/andivaanggoromachinelearning/blob/main/GoogleNET_Andiva_Kasih_Anggoro_Putra_1103204031.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Nama : Andiva Kasih Anggoro Putra

NIM : 1103204031

In [1]:
!pip install d2l

In [2]:
import torch
# library utama untuk komputasi tensor dan pembelajaran mendalam (deep learning) menggunakan PyTorch. Library ini menyediakan berbagai fungsi untuk operasi matematika dan manipulasi tensor.
from torch import nn
# digunakan untuk membangun dan melatih jaringan saraf, seperti lapisan jaringan saraf (layers), fungsi aktivasi, dan fungsi kerugian (loss functions).
import d2l.torch as d2l
# menyediakan berbagai fungsi dan utilitas yang membantu dalam pembelajaran mendalam menggunakan PyTorch. Modul ini mencakup fungsi untuk visualisasi, pemrosesan data, dan banyak lagi.
from torch.nn import functional as F
# berisi berbagai fungsi yang digunakan dalam membangun dan menjalankan jaringan saraf (neural networks). Fungsi-fungsi ini mencakup berbagai operasi yang sering digunakan dalam deep learning, seperti aktivasi, kehilangan (loss), dan operasi konvolusi.


In [3]:
class Inception(nn.Module):
    # c1--c4 adalah jumlah saluran keluaran untuk setiap cabang
    # Konstruktor Kelas dan Inisialisasi
    def __init__(self, c1, c2, c3, c4, **kwargs):
      # memanggil konstruktor dari kelas induk (nn.Module).
        super(Inception, self).__init__(**kwargs)

        # Branch 1
        self.b1_1 = nn.LazyConv2d(c1, kernel_size=1)
        # Branch pertama terdiri dari satu konvolusi dengan kernel 1x1.

        # Branch 2
        self.b2_1 = nn.LazyConv2d(c2[0], kernel_size=1)
        self.b2_2 = nn.LazyConv2d(c2[1], kernel_size=3, padding=1)
        # Branch kedua terdiri dari dua konvolusi berturut-turut: pertama dengan kernel 1x1, kemudian dengan kernel 3x3 (dengan padding 1).

        # Branch 3
        self.b3_1 = nn.LazyConv2d(c3[0], kernel_size=1)
        self.b3_2 = nn.LazyConv2d(c3[1], kernel_size=5, padding=2)
        # Branch ketiga terdiri dari dua konvolusi berturut-turut: pertama dengan kernel 1x1, kemudian dengan kernel 5x5 (dengan padding 2).

        # Branch 4
        self.b4_1 = nn.MaxPool2d(kernel_size=3, stride=1, padding=1)
        self.b4_2 = nn.LazyConv2d(c4, kernel_size=1)
        # Branch keempat terdiri dari pooling maksimal dengan kernel 3x3 (stride 1, padding 1), diikuti oleh konvolusi dengan kernel 1x1.

# forward mendefinisikan alur data melewati empat branch dan menggabungkan output dari semua branch dengan fungsi torch.cat pada dimensi channel.
    def forward(self, x):
        b1 = F.relu(self.b1_1(x))
        b2 = F.relu(self.b2_2(F.relu(self.b2_1(x))))
        b3 = F.relu(self.b3_2(F.relu(self.b3_1(x))))
        b4 = F.relu(self.b4_2(self.b4_1(x)))
        return torch.cat((b1, b2, b3, b4), dim=1)

In [4]:
# Membuat sebuah kelas bernama GoogleNet yang mewarisi (inherit) dari d2l.Classifier.
class GoogleNet(d2l.Classifier):
    def b1(self):
      # Membuat fungsi b1 di dalam kelas GoogleNet
        return nn.Sequential(
            # Mengembalikan sebuah Sequential container yang berisi beberapa lapisan neural network
            nn.LazyConv2d(64, kernel_size=7, stride=2, padding=3),
            #Konvolusi #2D #64Filter #Kernel7x7 #Stride2 #Padding3
            nn.ReLU(), # #AktivasiReLU
            nn.ReLU(), nn.MaxPool2d(kernel_size=3, stride=2, padding=1))
            #PoolingMaksimum #Kernel3x3 #Stride2 #Padding1


In [5]:
@d2l.add_to_class(GoogleNet)
# Baris ini kemungkinan merupakan bagian dari kerangka kerja deep learning (mungkin d2l) dan digunakan untuk mendaftarkan fungsi b2 sebagai metode di dalam kelas GoogleNet.
def b2(self):
  # # Definisi fungsi b2 yang dimasukkan ke dalam kelas GoogleNet
    return nn.Sequential(
        # Baris ini membuat wadah sekuensial yang akan menampung serangkaian lapisan jaringan saraf. Lapisan akan dieksekusi secara berurutan, dengan output dari satu lapisan dimasukkan ke input berikutnya.
        nn.LazyConv2d(64, kernel_size=1), nn.ReLU(),
        # Lapisan konvolusi dengan 64 filter, ukuran kernel 1x1 diikuti dengan fungsi aktivasi ReLU
        nn.LazyConv2d(192, kernel_size=3, padding=1), nn.ReLU(),
        # Lapisan konvolusi dengan 192 filter, ukuran kernel 3x3, padding 1 diikuti dengan fungsi aktivasi ReLU
        nn.MaxPool2d(kernel_size=3, stride=2, padding=1))
        # Lapisan max pooling dengan ukuran kernel 3x3, stride 2, dan padding 1


In [6]:
@d2l.add_to_class(GoogleNet)
# Mendefinisikan metode baru dalam kelas GoogleNet dari modul d2l
def b3(self):
  #  Parameter ini merujuk pada objek GoogleNet yang memanggil fungsi ini.

  # mengembalikan sebuah container urutan (nn.Sequential) yang akan menampung lapisan-lapisan jaringan saraf. Lapisan-lapisan ini akan dijalankan secara berurutan, dengan hasil dari satu lapisan menjadi input untuk lapisan berikutnya
    return nn.Sequential(Inception(64, (96, 128), (16, 32), 32),
                         Inception(128, (128, 192), (32, 96), 64),
                         nn.MaxPool2d(kernel_size=3, stride=2, padding=1))
                        #  Ini adalah lapisan Max Pooling 2D yang digunakan untuk mengurangi dimensi spasial input.


In [7]:
@d2l.add_to_class(GoogleNet)
# # Menambahkan metode b4 ke dalam kelas GoogleNet
def b4(self):
  # # Membuat sebuah Sequential model
    return nn.Sequential(Inception(192, (96, 208), (16, 48), 64),
                          # Inception module pertama dengan konfigurasi tertentu #Inception1
                         Inception(160, (112, 224), (24, 64), 64),
                         # Inception module kedua dengan konfigurasi tertentu #Inception2
                         Inception(128, (128, 256), (24, 64), 64),
                         # Inception module ketiga dengan konfigurasi tertentu #Inception3
                         Inception(112, (144, 288), (32, 64), 64),
                         # Inception module keempat dengan konfigurasi tertentu #Inception4
                         Inception(256, (160, 320), (32, 128), 128),
                         # Inception module kelima dengan konfigurasi tertentu #Inception5
                         nn.MaxPool2d(kernel_size=3, stride=2, padding=1))
                        # MaxPooling layer untuk mengurangi dimensi spasial #MaxPooling


In [8]:
@d2l.add_to_class(GoogleNet)
def b5(self):
    return nn.Sequential(Inception(256, (160, 320), (32, 128), 128),
                         #  # Inception module pertama dengan dimensi input dan filter yang ditentukan #InceptionPertama
                         Inception(384, (192, 384), (48, 128), 128),
                         # # Inception module kedua dengan dimensi input dan filter yang ditentukan #InceptionKedua
                         nn.AdaptiveAvgPool2d((1,1)), nn.Flatten())
                        # Lapisan pooling adaptif yang mengubah dimensi output menjadi (1, 1) #PoolingAdaptif


In [9]:
@d2l.add_to_class(GoogleNet)
# Menggunakan dekorator untuk menambahkan metode ke kelas GoogleNet
def __init__(self, lr=0.1, num_classes=10):
  #Inisialisasi kelas dengan parameter default untuk laju pembelajaran dan jumlah kelas
    super(GoogleNet, self).__init__()
    #Memanggil konstruktor kelas induk dari nn.Module
    self.save_hyperparameters()
    # #Menyimpan hyperparameter untuk digunakan kemudian
    self.net = nn.Sequential(self.b1(), self.b2(), self.b3(), self.b4(),
                             self.b5(), nn.LazyLinear(num_classes))
    #Membuat model berurutan dengan lima blok dan satu lapisan linear untuk klasifikasi
    self.net.apply(d2l.init_cnn)
    # #Menerapkan inisialisasi ke semua lapisan jaringan


In [10]:
model = GoogleNet().layer_summary((1, 1, 96, 96))
# Pemanggilan metode ini mencetak ringkasan lapisan model, termasuk namanya, bentuk masukan dan keluaran, serta jumlah parameter.


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/lazy.py:181: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


Sequential output shape:	 torch.Size([1, 64, 24, 24])
Sequential output shape:	 torch.Size([1, 192, 12, 12])
Sequential output shape:	 torch.Size([1, 480, 6, 6])
Sequential output shape:	 torch.Size([1, 832, 3, 3])
Sequential output shape:	 torch.Size([1, 1024])
Linear output shape:	 torch.Size([1, 10])


In [ ]:
model = GoogleNet(lr=0.01)
# membuat sebuah instance dariGoogleNetmodel,kemungkinan dari perpustakaan pembelajaran mendalam (d2l diasumsikan di sini).
# Ini juga menetapkan kecepatan pembelajaran (lr) ke 0.01,yang mengontrol seberapa banyak model memperbarui bobotnya selama pelatihan.
trainer = d2l.Trainer(max_epochs=10, num_gpus=1)
# membuat objek pelatih,kemungkinan dari perpustakaan yang sama.Pelatih mengelola proses pelatihan,termasuk melacak zaman,iterasi,dan memperbarui bobot model.
data = d2l.FashionMNIST(batch_size=128, resize=(96, 96))
# memuat kumpulan data FashionMNIST,kumpulan data yang umum digunakan untuk tugas klasifikasi gambar
model.apply_init([next(iter(data.get_dataloader(True)))[0]], d2l.init_cnn)
# menginisialisasi bobot model dengan nilai yang sesuai untuk pelatihan.Rincian spesifik darid2l.init_cnnmungkin bergantung pada perpustakaan,tetapi umumnya menetapkan bobot ke nilai acak atau menggunakan strategi inisialisasi yang telah ditentukan sebelumnya yang sesuai untuk jaringan saraf konvolusional.
trainer.fit(model, data)
# memulai proses pelatihan.Objek pelatih mengambil model dan kumpulan data sebagai masukan dan melakukan tindakan


100%|██████████| 26421880/26421880 [00:04<00:00, 6506637.63it/s] 


Extracting ../data/FashionMNIST/raw/train-images-idx3-ubyte.gz to ../data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 126843.83it/s]


Extracting ../data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ../data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:08<00:00, 498412.28it/s] 


Extracting ../data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ../data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 4884025.56it/s]

Extracting ../data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/FashionMNIST/raw




/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


KeyboardInterrupt: 